# Gestion du cache

Le principe d'un cache est de conserver les résultats de requêtes pour ne pas avoir à les recalculer lorsque ces requêtes seront à nouveau reçues. Le cache est intéressant à partir du moment où des requêtes reviennent assez fréquemment, sachant qu'en contrepartie, la cache va consommer de la ressource mémoire, disque et CPU. Il faut que cette consommation de ressources soit plus rentable que la consommation faite par le calcul qu'implique la requête reçue, pour que le cache soit considéré comme une option intéressante.

Spark propose une fonctionnalité de cache au niveau Spark Core sur les RDD et Spark SQL sur les dataframe et les dataset. Dans le cadre de Spark SQL, le cache devient intéressant dès qu'un dataframe est utilisé dans plusieurs requêtes séparées. Dans cette situation, l'utilisation du cache pour un dataframe permet au niveau de chacune des requêtes qui utilisent ce dataframe de réévaluer à chaque fois les calculs qui ont conduit à ce dataframe.

## Prélude

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = (SparkSession.builder
    .appName("Spark tuning - Cache")
    .master("local[*]")
    .config("spark.ui.showConsoleProgress", "True")
    .config("spark.executor.memory", "2g")
    .getOrCreate())

# Access the JVM and import the required Java classes
jvm = spark.sparkContext._jvm
Level = jvm.org.apache.logging.log4j.Level
Configurator = jvm.org.apache.logging.log4j.core.config.Configurator

# Set the root level to OFF
Configurator.setRootLevel(Level.OFF)

## Introduction sur le cache

In [3]:
orders = (spark.read
    .option("header", "true")
    .csv("data/orders.csv")
    .repartition(4))

orders.printSchema()
orders.show(10, False)

root
 |-- id: string (nullable = true)
 |-- client: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- product: string (nullable = true)
 |-- price: string (nullable = true)

+--------+----------------+-------------------+------------+-----+
|id      |client          |timestamp          |product     |price|
+--------+----------------+-------------------+------------+-----+
|98432797|oplTx8h-38G3be4c|2023-02-27T10:34:31|décaféiné   |1.4  |
|90609564|D7pVaSr2-BpeKGwE|2023-03-02T17:59:04|café crème  |2.5  |
|03804834|D7pVaSr2-BpeKGwE|2022-12-19T10:50:17|double café |2.6  |
|23879812|t_CUBr6tyTQxGj2X|2023-02-03T16:29:14|décaféiné   |1.4  |
|73975031|TX7wC0pTqCRlCOhi|2023-04-20T09:43:28|café allongé|1.4  |
|15380364|oplTx8h-38G3be4c|2023-01-25T16:38:28|expresso    |1.1  |
|25138740|oplTx8h-38G3be4c|2023-05-01T17:37:46|noisette    |1.5  |
|57293732|TX7wC0pTqCRlCOhi|2023-05-04T09:27:31|double café |2.6  |
|98462522|H-Mp22FLe99MNhRa|2023-03-24T10:08:22|expresso    |1.1  |
|

### Sans cache

Nous voulons effectuer des analyses sur la journée du 1er février 2023, sans utiliser de cache.

In [5]:
partOfOrders = orders.where(to_date(col("timestamp")) == to_date(lit("2023-02-01")))

partOfOrders.show(10, False)

+--------+----------------+-------------------+--------------+-----+
|id      |client          |timestamp          |product       |price|
+--------+----------------+-------------------+--------------+-----+
|90763431|t_CUBr6tyTQxGj2X|2023-02-01T17:19:26|chocolat chaud|2.6  |
|38930443|XztHU0aeUckvR7AC|2023-02-01T13:17:59|café crème    |2.5  |
|27370329|iQ5y--CyNtHUDL_8|2023-02-01T10:37:08|chocolat chaud|2.6  |
|69412681|JBoCs7rWb_jEs87W|2023-02-01T09:57:30|expresso      |1.1  |
|24134740|JBoCs7rWb_jEs87W|2023-02-01T17:58:08|café          |1.3  |
|52156742|vAfh79KyDpYeFEMR|2023-02-01T13:02:35|double café   |2.6  |
|26805451|D7pVaSr2-BpeKGwE|2023-02-01T13:47:13|double café   |2.6  |
|33619683|iQ5y--CyNtHUDL_8|2023-02-01T09:19:38|café          |1.3  |
|17099792|H-Mp22FLe99MNhRa|2023-02-01T10:22:44|café allongé  |1.4  |
|77346060|XztHU0aeUckvR7AC|2023-02-01T16:48:48|café crème    |2.5  |
+--------+----------------+-------------------+--------------+-----+
only showing top 10 rows



Nous récupérons d'abord le chiffre d'affaires de la journée.

In [27]:
# show revenue - no cache
partOfOrders.agg(round(sum(col("price")), 2).alias("total")).show()

+-----+
|total|
+-----+
|119.2|
+-----+



Nous allons ensuite analyser la répartition du chiffre d'affaires de la journée sur les différents produits.

In [12]:
# show product revenue - no cache
(partOfOrders
.groupBy("product")
.agg(sum("price").alias("total"))
.where("total >= 15.0")
.orderBy(desc("total"))
.show())

+-----------+------------------+
|    product|             total|
+-----------+------------------+
|   expresso|              23.1|
| café crème|              17.5|
|double café|15.600000000000001|
+-----------+------------------+



Nous récupérons enfin la répartition des ventes par client sur le café noisette et le total des ventes de la journée sur ce produit.

In [13]:
# show noisette - no cache
(partOfOrders
.where(col("product") == "noisette")
.cube("client")
.agg(sum("price").alias("total"))
.orderBy(desc("client"))
.show())

+----------------+-----+
|          client|total|
+----------------+-----+
|vAfh79KyDpYeFEMR|  3.0|
|oplTx8h-38G3be4c|  1.5|
|JBoCs7rWb_jEs87W|  1.5|
|H-Mp22FLe99MNhRa|  1.5|
|D7pVaSr2-BpeKGwE|  3.0|
|            null| 10.5|
+----------------+-----+



👀 **Question** 👀

Dans toutes ces requêtes, lorsque nous regardons le plan d'exécution, nous voyons des DAG relativement équivalents. De plus chaque requête se traduit par l'exécution de 3 jobs. Nous le voyons dans ce notebook, au niveau des barres de progression.

### Avec cache

Nous allons maintenant exécuter les mêmes requêtes. Mais cette fois, nous allons mettre en cache le dataframe filtré sur la journée 1er février 2023.

In [14]:
partOfOrders = orders.where(to_date("timestamp") == to_date(lit("2023-02-01"))).cache()

partOfOrders.show(10, False)

+--------+----------------+-------------------+--------------+-----+
|id      |client          |timestamp          |product       |price|
+--------+----------------+-------------------+--------------+-----+
|90763431|t_CUBr6tyTQxGj2X|2023-02-01T17:19:26|chocolat chaud|2.6  |
|38930443|XztHU0aeUckvR7AC|2023-02-01T13:17:59|café crème    |2.5  |
|27370329|iQ5y--CyNtHUDL_8|2023-02-01T10:37:08|chocolat chaud|2.6  |
|69412681|JBoCs7rWb_jEs87W|2023-02-01T09:57:30|expresso      |1.1  |
|24134740|JBoCs7rWb_jEs87W|2023-02-01T17:58:08|café          |1.3  |
|52156742|vAfh79KyDpYeFEMR|2023-02-01T13:02:35|double café   |2.6  |
|26805451|D7pVaSr2-BpeKGwE|2023-02-01T13:47:13|double café   |2.6  |
|33619683|iQ5y--CyNtHUDL_8|2023-02-01T09:19:38|café          |1.3  |
|17099792|H-Mp22FLe99MNhRa|2023-02-01T10:22:44|café allongé  |1.4  |
|77346060|XztHU0aeUckvR7AC|2023-02-01T16:48:48|café crème    |2.5  |
+--------+----------------+-------------------+--------------+-----+
only showing top 10 rows



👷 Exécutez les requêtes ci-dessous.

In [16]:
# show revenue - cache
(
    partOfOrders
        .agg(round(sum("price"), 2).alias("total"))
        .show()
)

+-----+
|total|
+-----+
|119.2|
+-----+



In [17]:
# show product revenue - cache
(
  partOfOrders
    .groupBy("product")
    .agg(sum("price").alias("total"))
    .where(col("total") >= 15.0)
    .orderBy(desc("total"))
    .show()
)

+-----------+------------------+
|    product|             total|
+-----------+------------------+
|   expresso|              23.1|
| café crème|              17.5|
|double café|15.600000000000001|
+-----------+------------------+



In [18]:
# show noisette - cache
(
  partOfOrders
    .where(col("product") == "noisette")
    .cube("client")
    .agg(sum("price").alias("total"))
    .orderBy(desc("client"))
    .show()
)

+----------------+-----+
|          client|total|
+----------------+-----+
|vAfh79KyDpYeFEMR|  3.0|
|oplTx8h-38G3be4c|  1.5|
|JBoCs7rWb_jEs87W|  1.5|
|H-Mp22FLe99MNhRa|  1.5|
|D7pVaSr2-BpeKGwE|  3.0|
|            null| 10.5|
+----------------+-----+



👀 **Ce qu'il faut voir** 👀

La différence, qui apparaît ici, est que l'exécution des requêtes d'analyse nécessite cette 2 jobs au lieu de 3. Ce qui permet un gain en performance sur l'exécution de ces requêtes.

En regardant de plus près le plan d'exécution de ces requêtes, nous remarquons que celui-ci est un plus grand. Dans le cadre de la mise en cache, Spark a ajouté les étapes :

* **InMemoryRelation** : mise en cache des données du dataframe
* **InMemoryTableScan** : parcours des données du dataframe depuis le cache

Ces étapes proviennent de `partOfOrders`.

=> 👷 Exécutez la cellule suivante pour voir le plan d'exécution lié à `partOfOrders`.

In [19]:
partOfOrders.explain()

== Physical Plan ==
InMemoryTableScan [id#17, client#18, timestamp#19, product#20, price#21]
   +- InMemoryRelation [id#17, client#18, timestamp#19, product#20, price#21], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- Exchange RoundRobinPartitioning(4), REPARTITION_BY_NUM, [plan_id=1031]
            +- *(1) Filter (isnotnull(timestamp#19) AND (cast(timestamp#19 as date) = 2023-02-01))
               +- FileScan csv [id#17,client#18,timestamp#19,product#20,price#21] Batched: false, DataFilters: [isnotnull(timestamp#19), (cast(timestamp#19 as date) = 2023-02-01)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/work/04_advanced/data/orders.csv], PartitionFilters: [], PushedFilters: [IsNotNull(timestamp)], ReadSchema: struct<id:string,client:string,timestamp:string,product:string,price:string>




En fait, dans les requêtes d'analyse, toutes les étapes précédentes _InMemoryTableScan_ ne sont pas exécutées.

Si vous allez dans Spark UI, dans l'onglet _Storage_ vous allez trouver un tableau avec une seule entrée, contenant les informations ci-dessous :

* Storage Level: Disk Memory Deserialized 1x Replicated
* Cached Partitions: 4
* Fraction Cached: 100%
* Size in Memory: 8.6 KiB
* Size on Disk: 0.0 B

Cette ligne unique indique les informations sur les données en cache liées à `partOfOrders`.

_Storage Level_ indique le type de stockage utilisé. Ici, nous avons un stockage en mémoire désérialisé (il s'agit uniquement d'objets Java stockés dans la _heap_ de la JVM) et sur disque, avec un unique répliqua (il n'y a pas de réplication sur d'autres executor).

Nous avons aussi la quantité de mémoire utilisée et la quantité de données stockée sur disque.

Si vous cliquez sur la ligne du tableau dans Spark UI, vous obtenez plus d'informations sur nos données en cache. Nous avons notamment la répartition des données en cache par partition avec les executor associés. Et nous avons aussi la mémoire restante pour le cache.

Il est possible de supprimer le cache à la main en utilisant la méthode `.unpersist()`.

In [20]:
partOfOrders.unpersist()

DataFrame[id: string, client: string, timestamp: string, product: string, price: string]

Si vous retournez dans l'onglet _Storage_ de Spark UI, vous verrez que le tableau aura disparu.

### Fonctionnement

Le cache Spark SQL est un cache LRU (_Least Recently Used_). Ainsi, lorsque l'espace pour le cache manque, toutes les entrées les moins utilisées sont retirées du cache pour laisser de la place. Lors du prochain accès aux données retirées, Spark recalculera ces données pour les réintégrer dans le cache, en ayant au préalable retiré les données les moins utilisées, si le cache est à nouveau plein.

## Les différents niveaux de cache

### CacheManager

Spark SQL bénéficie d'un CacheManager qui centralise la gestion des caches.

Il est possible de vider complètement le cache à travers son interface.

In [24]:
cache_manager = spark._jsparkSession.sharedState().cacheManager()

cache_manager.clearCache()

Voyons le fonctionnement du CacheManager. Pour cela, nous allons mettre en cache 2 requêtes identiques, mais réclamant une stratégie du stockage du cache différente.

In [28]:
orders_default = spark.read.option("header", "true").csv("data/orders.csv").repartition(4).cache()
orders_default.take(1)

[Row(id='98432797', client='oplTx8h-38G3be4c', timestamp='2023-02-27T10:34:31', product='décaféiné', price='1.4')]

In [30]:
from pyspark import StorageLevel
orders_memory = spark.read.option("header", "true").csv("data/orders.csv").repartition(4).persist(StorageLevel.MEMORY_ONLY)
orders_default.take(1)

[Row(id='98432797', client='oplTx8h-38G3be4c', timestamp='2023-02-27T10:34:31', product='décaféiné', price='1.4')]

👀 **Ce qu'il faut voir** 👀

Si nous regardons dans Spark UI, bien que nous ayons produits deux dataframe, nous ne verrons qu'un seul cache dans l'onglet _Storage_.

Plus exactement, la recherche du cache se fait selon l'opération suivante :

In [34]:
# _jdf accesses the underlying Java DataFrame from the PySpark DataFrame
orders_default._jdf.queryExecution().logical().sameResult(orders_memory._jdf.queryExecution().logical())

True

Ce qui veut dire que ce qui est retenu, c'est plus exactement la similarité des résultats obtenus par les dataframes et non pas sur l'égalité entre les plans d'exécution des dataframes.

La difficulté est que tout ceci est déterminé avant l'exécution de la requête. Il est dans ce cas parfois difficile de déterminer si deux plans d'exécution produiront des résultats équivalents. Spark ne sait pas toujours le déterminer. Par contre, Spark va capable de déterminer si deux plans d'exécution vont donner des résultats différents.

Pour retrouver des données en cache, Spark SQL passe par la méthode `.lookupCachedData()` du CacheManager.

In [35]:
class TestUniqueCache:
    def __init__(self):
        self.cacheManager = spark._jsparkSession.sharedState().cacheManager()
        self.cache_orders_default = self.cacheManager.lookupCachedData(orders_default._jdf).get()
        self.cache_orders_memory = self.cacheManager.lookupCachedData(orders_memory._jdf).get()

    def run(self):
        same_result = self.cache_orders_default.plan().sameResult(self.cache_orders_memory.plan())
        print(f"cache_orders_default.plan sameResult cache_orders_memory.plan: {same_result}")
        
        storage_level_default = self.cache_orders_default.cachedRepresentation().cacheBuilder().storageLevel()
        print(f"cache_orders_default: {storage_level_default}")
        
        storage_level_memory = self.cache_orders_memory.cachedRepresentation().cacheBuilder().storageLevel()
        print(f"cache_orders_memory: {storage_level_memory}")

TestUniqueCache().run()

cache_orders_default.plan sameResult cache_orders_memory.plan: True
cache_orders_default: StorageLevel(disk, memory, deserialized, 1 replicas)
cache_orders_memory: StorageLevel(disk, memory, deserialized, 1 replicas)


D'ailleurs, notre dataframe d'origine est lui aussi indexé dans le cache, même si son plan d'exécution déduit n'a pas changé.

In [41]:
orders.explain()
cacheManager = spark._jsparkSession.sharedState().cacheManager()
cacheManager.lookupCachedData(orders._jdf).isDefined()

== Physical Plan ==
InMemoryTableScan [id#17, client#18, timestamp#19, product#20, price#21]
   +- InMemoryRelation [id#17, client#18, timestamp#19, product#20, price#21], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- Exchange RoundRobinPartitioning(4), REPARTITION_BY_NUM, [plan_id=1334]
            +- FileScan csv [id#1015,client#1016,timestamp#1017,product#1018,price#1019] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/work/04_advanced/data/orders.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<id:string,client:string,timestamp:string,product:string,price:string>




True

Si vous exécutez la cellule ci-dessous et que vous allez dans Spark UI, vous verrez apparaître une étape InMemoryRelation et InMemoryTableScan, indiquant que les données pour cette requête n'ont pas été récupérées depuis le fichier, mais depuis le cache.

In [42]:
# take from order - before clear cache
orders.take(1)

[Row(id='98432797', client='oplTx8h-38G3be4c', timestamp='2023-02-27T10:34:31', product='décaféiné', price='1.4')]

Par contre, si nous vidons le cache, la récupération des données se fait depuis le fichier directement. Dans le plan d'exécution, dans Spark UI, les étapes InMemoryRelation et InMemoryTableScan auront disparues.

In [46]:
cacheManager.clearCache()

orders.explain()
print(f"orders defined ? {cacheManager.lookupCachedData(orders._jdf).isDefined()}")

# take from order - after clear cache
orders.take(1)

== Physical Plan ==
InMemoryTableScan [id#17, client#18, timestamp#19, product#20, price#21]
   +- InMemoryRelation [id#17, client#18, timestamp#19, product#20, price#21], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- Exchange RoundRobinPartitioning(4), REPARTITION_BY_NUM, [plan_id=1334]
            +- FileScan csv [id#1015,client#1016,timestamp#1017,product#1018,price#1019] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/work/04_advanced/data/orders.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<id:string,client:string,timestamp:string,product:string,price:string>


orders defined ? False


[Row(id='98432797', client='oplTx8h-38G3be4c', timestamp='2023-02-27T10:34:31', product='décaféiné', price='1.4')]

### Stockage des caches

Voici la liste des stratégies de stockage proposée par Spark.

* **NONE** : pas de persistance (empêche le CacheManager de persister par la suite un type de requête)
* **DISK_ONLY** : persistance sur le disque local
* **DISK_ONLY_2** : persistance sur disque répliquée sur 2 nœuds
* **DISK_ONLY_3** : persistance sur disque répliquée sur 3 nœuds
* **MEMORY_ONLY** : persistance en mémoire
* **MEMORY_ONLY_2** : persistance en mémoire sur 2 nœuds
* **MEMORY_ONLY_SER** : persistance sérialisée en mémoire
* **MEMORY_ONLY_SER_2** : persistance sérialisée en mémoire sur 2 nœuds
* **MEMORY_AND_DISK** : persistance en mémoire et sur disque
* **MEMORY_AND_DISK_2** : persistance en mémoire et sur disque répliquée sur 2 nœuds
* **MEMORY_AND_DISK_SER** : persistance sérialisée en mémoire et sur disque
* **MEMORY_AND_DISK_SER_2** : persistance sérialisée en mémoire et sur disque répliquée sur 2 nœuds
* **OFF_HEAP** : persistance en mémoire _off heap_ (_expériemntal_)

La sérialisation proposée ici utilise un encodage binaire propre à Spark et qui reconnu comme efficace. Il permet de gagner de l'espace en mémoire, en demandant un effort supplémentaire sur le CPU.

Le stockage sur disque est bien évidemment plus coûteux que le stockage en mémoire. Il faut dans ce cas que le recalcul du dataframe soit plus lent que l'accès disque pour récupérer les données.

La réplication est intéressante si le recalcul du dataframe est extrêmement lent et que l'on souhaite donner la possibilité à Spark de reprendre les traitements en cours sur un autre executor, en cas de panne partielle.

**Exercice**

👷 Ci-dessous, après avoir vidé le cache, vérifier dans Spark UI l'effet du stockage `NONE`.

In [47]:
cacheManager.clearCache()

In [48]:
# StorageLevel.NONE
orders.where(to_date("timestamp") == to_date(lit("2023-02-28"))).persist(StorageLevel.NONE).take(1)

[Row(id='56391935', client='JBoCs7rWb_jEs87W', timestamp='2023-02-28T10:45:14', product='noisette', price='1.5')]

👷 Exécutez les cellules ci-dessous et observez les différences. A noter que [certaines méthodes de sérialisation ne sont pas disponibles en PySpark](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence) (ie: MEMORY_ONLY_SER / MEMORY_AND_DISK_SER)

In [52]:
#StorageLevel.MEMORY_ONLY
memory_only = orders.where(to_date("timestamp") == to_date(lit("2023-03-01"))).persist(StorageLevel.MEMORY_ONLY).take(1)
print(f"memory_only = {memory_only}")

memory_only = [Row(id='49405607', client='H-Mp22FLe99MNhRa', timestamp='2023-03-01T16:57:01', product='café crème', price='2.5')]


In [55]:
#StorageLevel.DISK_ONLY
disk_only = orders.where(to_date("timestamp") == to_date(lit("2023-03-01"))).persist(StorageLevel.DISK_ONLY).take(1)
print(f"disk_only = {disk_only}")

disk_only = [Row(id='49405607', client='H-Mp22FLe99MNhRa', timestamp='2023-03-01T16:57:01', product='café crème', price='2.5')]


In [56]:
#StorageLevel.MEMORY_AND_DISK
memory_and_disk = orders.where(to_date("timestamp") == to_date(lit("2023-03-01"))).persist(StorageLevel.MEMORY_AND_DISK).take(1)
print(f"memory_and_disk = {memory_and_disk}")

memory_and_disk = [Row(id='49405607', client='H-Mp22FLe99MNhRa', timestamp='2023-03-01T16:57:01', product='café crème', price='2.5')]


In [57]:
#experimental
#StorageLevel.OFF_HEAP
off_heap = orders.where(to_date("timestamp") == to_date(lit("2023-03-01"))).persist(StorageLevel.OFF_HEAP).take(1)
print(f"off_heap = {off_heap}")

off_heap = [Row(id='49405607', client='H-Mp22FLe99MNhRa', timestamp='2023-03-01T16:57:01', product='café crème', price='2.5')]


🤔 **Question** 🤔

Vérifiez dans Spark UI la stratégie de stockage qui semble la meilleure. Appuyez-vous pour cela sur les statistiques de latence et de consommation d'espace.

----

Nous allons maintenant voir l'effet d'une stratégie de stockage impliquant une réplication, sachant que nous n'avons qu'un seul executor (ie. le driver).

Après avoir vidé le cache et lancé la requête, regardez dans Spark UI, dans l'onglet _Storage_, si un espace de stockage a été créé.

In [49]:
cacheManager.clearCache()
#StorageLevel.DISK_ONLY_2
orders.where(to_date("timestamp") == to_date(lit("2023-03-07"))).persist(StorageLevel.DISK_ONLY_2).take(1)

[Row(id='16899776', client='iQ5y--CyNtHUDL_8', timestamp='2023-03-07T09:28:32', product='noisette', price='1.5')]